# QC Handbook 
## Diamond 2:
### Implementations:
- QC on the preprocessed data
	- Check carpet plots for preprocessed subjects
	- Were the 1stlevels run for all tasks
	- What contrasts/PPI were run

### QC Implementations to figure out which subjects should be removed
- Get motion pars for each subject (6 motion 1 FD)
    - calculate min/max/avg
    - Use specific exclusion value for each par
        - Which tasks have unasable motion?
        - What value did it have?
- Run percent coverage script
    - If value is below certain percent threshold, output that ROI
    - If vital areas have dropout, exclude that task
    - What areas are most vital?
- Check to make sure time series was completely collected for task
    - How many timepoints should there be for the task?
        - If it has less than the required timepoints, output the error and exlude that task
|

## QC on the RAWDATA
- Time Series correct length
- Serious motion check
    - Alex/Tyler could build a tool to look at this effectively
        - The tool should be able to be used by the data manager
- Serious dropout check
- Was Physio data collected
- Are the behavioral files present/complete for all tasks

In [1]:
import sys
sys.path.insert(1, 'jupyter-notebooks')
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import glob
import os
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [2]:
file1 = 'dicominfo_TSVs/50002_dicominfo_ses-1.txt'
file2 = 'dicominfo_TSVs/50004_dicominfo_ses-1.txt'
files = glob.glob('dicominfo_TSVs/*dicominfo_ses-1.txt')
df1 = pd.read_csv(file1, sep='\t')
df2 = pd.read_csv(file2, sep='\t')
df1['subject_name'] = os.path.basename(file1).split('_')[0]
df2['subject_name'] = os.path.basename(file2).split('_')[0]

FileNotFoundError: [Errno 2] No such file or directory: 'dicominfo_TSVs/50002_dicominfo_ses-1.txt'

In [3]:
results_df = pd.DataFrame()
for x in files:
    df = pd.read_csv(x, sep='\t')
    df['subject_name'] = os.path.basename(x).split('_')[0]
    query = "SELECT subject_name,protocol_name, COUNT(protocol_name) AS cnt FROM df GROUP BY subject_name, protocol_name"
    current_results = pysqldf(query)
    results_df = results_df.append(current_results)

In [4]:
import sys
sys.path

['/data/D2/data',
 'jupyter-notebooks',
 '/usr/local/software/environments/QC/lib/python38.zip',
 '/usr/local/software/environments/QC/lib/python3.8',
 '/usr/local/software/environments/QC/lib/python3.8/lib-dynload',
 '',
 '/home/skebaas/.local/lib/python3.8/site-packages',
 '/home/skebaas/.local/lib/python3.8/site-packages/waipy-0.0.1.51-py3.8.egg',
 '/usr/local/software/environments/QC/lib/python3.8/site-packages']

In [5]:
results_df.set_index('subject_name').sort_index()

KeyError: "None of ['subject_name'] are in the columns"

## The QC_Pipeline module functionality
### The QC_Pipeline module can generate the following information:
- `task_df`: (DataFrame) Table containing mean/max metrics for 6 motion_pars and Framewise_displacement
- `mean_table`: (DataFrame) Table containing only values that have been flagged with mean value > `0.5`
- `max_table`: (DataFrame) Table containing only values that have been flagged with max value > `5`
- `mean_table_highlighted`: (StyleFrame) Table containing same values as `mean_table` but with the flagged values highlighted
- `max_table_highlighted`: (StyleFrame) Table containing same values as `max_table` but with the flagged values highlighted
- `mean_exclued`: (Integer) Number of subjects that were flagged for the mean
- `max_excluded`: (Integer) Number of subjects that were flagged for the max
- `percent_coverage_table`: (DataFrame) Table containing the percent coverage found for each ROI present in the specified atlas
- `percent_coverage_highlighted`: (StyleFrame) Same table as `percent_coverage_table` but with ROIs where subject has less than 20% coverage highlighted

## Example 1: Review QC for Dynamic Faces data

In [7]:
from QC_Pipeline import Task_DataFrame as QC #Import custom module for QC
dynface = QC('dynface', 'Preprocessed_ICA/NON/') #specify task and folder of fmriprep data

dynface and Preprocessed_ICA/NON/


In [8]:
#display the generated tables
#print('task_df')
#display(reward1_NON.task_df.sort_index()) #Jupyter function to display a DataFrame/StyleFrame
print('mean_table')
display(dynface.mean_table.sort_index())
print('max_table')
display(dynface.max_table.sort_index())
print('mean_table_highlighted')
display(dynface.mean_table_highlighted)
print('max_table_highlighted')
display(dynface.max_table_highlighted)
print(f'mean_excluded for {dynface.task} is {dynface.mean_excluded}')
print(f'max_excluded for {dynface.task} is {dynface.max_excluded}')

mean_table


,trans_x_mean,trans_x_max,trans_y_mean,trans_y_max,trans_z_mean,trans_z_max,rot_x_mean,rot_x_max,rot_y_mean,rot_y_max,rot_z_mean,rot_z_max,framewise_displacement_mean,framewise_displacement_max
sub-50002,0.023021,0.181545,0.028462,0.451141,0.281657,3.664040,-0.010484,0.000978,-0.001344,0.000998,-0.000426,0.001289,0.250766,7.425808
sub-50004,0.202583,2.416030,0.191919,3.070580,0.648134,3.295040,-0.007105,0.034273,-0.014217,0.004143,0.004132,0.046028,0.402640,11.536665
sub-50007,-0.880137,0.307591,3.710501,17.991200,5.513927,15.747900,0.115527,0.450152,0.022132,0.052565,0.005142,0.019473,0.707150,26.568369
sub-50009,0.153773,1.058900,0.341077,0.784399,0.861241,4.093070,0.001649,0.041608,-0.001201,0.010181,-0.001932,0.004210,0.286060,6.079379
sub-50018,-0.404670,0.026091,-0.967570,0.224384,-0.946846,0.469011,-0.031094,0.033347,0.008172,0.019616,0.009199,0.027136,0.230944,9.776497
sub-50054,-0.677896,0.995117,-0.116753,1.858980,1.267442,5.250500,-0.009445,0.073285,0.014230,0.073829,-0.020413,0.008628,1.358514,10.853451
sub-50070,-0.110707,0.174075,0.215738,2.179960,-0.184914,2.420050,0.009364,0.088542,0.004744,0.014025,-0.004118,0.000802,0.217867,6.433287
sub-50108,0.055117,4.402010,2.019875,6.076770,5.424303,11.858500,-0.004006,0.108425,-0.005190,0.051852,0.009673,0.052293,0.907316,21.546342
sub-50117,0.078564,1.813200,1.051401,5.966390,3.452301,10.611100,0.020457,0.249203,-0.007240,0.008221,0.004451,0.080002,0.845632,16.320245
sub-50122,-0.212061,0.005181,-0.020354,1.915960,-0.593429,2.044270,0.003613,0.093633,-0.006839,0.018784,-0.003048,0.001704,0.250789,6.655741


max_table


,trans_x_mean,trans_x_max,trans_y_mean,trans_y_max,trans_z_mean,trans_z_max,rot_x_mean,rot_x_max,rot_y_mean,rot_y_max,rot_z_mean,rot_z_max,framewise_displacement_mean,framewise_displacement_max
sub-50004,0.202583,2.416030,0.191919,3.070580,0.648134,3.295040,-0.007105,0.034273,-0.014217,0.004143,0.004132,0.046028,0.402640,11.536665
sub-50007,-0.880137,0.307591,3.710501,17.991200,5.513927,15.747900,0.115527,0.450152,0.022132,0.052565,0.005142,0.019473,0.707150,26.568369
sub-50009,0.153773,1.058900,0.341077,0.784399,0.861241,4.093070,0.001649,0.041608,-0.001201,0.010181,-0.001932,0.004210,0.286060,6.079379
sub-50014,-0.083381,0.103778,0.164474,0.656476,0.608015,2.431500,-0.002186,0.013528,-0.004929,0.000345,-0.002205,0.003337,0.180141,2.385506
sub-50051,0.095784,0.179959,0.516582,1.054990,0.655904,0.978345,-0.001843,0.022533,0.007489,0.016175,-0.004443,0.000739,0.148341,1.116185
sub-50054,-0.677896,0.995117,-0.116753,1.858980,1.267442,5.250500,-0.009445,0.073285,0.014230,0.073829,-0.020413,0.008628,1.358514,10.853451
sub-50083,-0.010054,0.030299,0.110041,0.378917,0.614608,1.013700,-0.001971,0.002376,-0.003093,-0.000169,0.001561,0.002415,0.186041,1.078748
sub-50084,-0.437650,-0.037310,-0.022163,0.140955,0.610585,1.026470,0.002675,0.005707,-0.000695,0.001962,-0.001038,0.002490,0.109104,0.508112
sub-50106,0.079223,0.201422,0.277091,0.545240,0.525200,0.995413,0.003529,0.010794,0.000352,0.002853,0.005018,0.009318,0.119638,1.025588
sub-50108,0.055117,4.402010,2.019875,6.076770,5.424303,11.858500,-0.004006,0.108425,-0.005190,0.051852,0.009673,0.052293,0.907316,21.546342


mean_table_highlighted


,trans_x_mean,trans_y_mean,trans_z_mean,rot_x_mean,rot_y_mean,rot_z_mean,framewise_displacement_mean
sub-50106,0.079223,0.277091,0.525200,0.003529,0.000352,0.005018,0.119638
sub-50051,0.095784,0.516582,0.655904,-0.001843,0.007489,-0.004443,0.148341
sub-50149,-0.064744,-0.018431,1.483771,-0.017496,0.003602,-0.002077,0.365048
sub-50113,0.000500,0.174860,0.549654,-0.000401,0.002428,-0.001622,0.074572
sub-50156,-0.005514,0.860847,0.126397,0.012811,-0.001266,-0.000485,0.067486
sub-50141,0.425798,0.765321,0.769714,0.028395,-0.002938,0.004441,0.141937
sub-50115,-0.058352,0.553930,0.859283,-0.001849,0.005313,0.001238,0.146453
sub-50108,0.055117,2.019875,5.424303,-0.004006,-0.005190,0.009673,0.907316
sub-50054,-0.677896,-0.116753,1.267442,-0.009445,0.014230,-0.020413,1.358514
sub-50175,-0.094459,0.105197,0.529811,-0.005589,-0.002313,0.000800,0.088886


max_table_highlighted


,trans_x_max,trans_y_max,trans_z_max,rot_x_max,rot_y_max,rot_z_max,framewise_displacement_max
sub-50070,0.174075,2.179960,2.420050,0.088542,0.014025,0.000802,6.433287
sub-50149,1.007490,0.404427,2.925430,0.017602,0.014443,0.012744,7.067934
sub-50108,4.402010,6.076770,11.858500,0.108425,0.051852,0.052293,21.546342
sub-50054,0.995117,1.858980,5.250500,0.073285,0.073829,0.008628,10.853451
sub-50007,0.307591,17.991200,15.747900,0.450152,0.052565,0.019473,26.568369
sub-50153,0.193355,0.645485,0.850721,0.009074,0.007427,0.004543,5.942512
sub-50117,1.813200,5.966390,10.611100,0.249203,0.008221,0.080002,16.320245
sub-50018,0.026091,0.224384,0.469011,0.033347,0.019616,0.027136,9.776497
sub-50122,0.005181,1.915960,2.044270,0.093633,0.018784,0.001704,6.655741
sub-50009,1.058900,0.784399,4.093070,0.041608,0.010181,0.004210,6.079379


mean_excluded for dynface is 12
max_excluded for dynface is 26


In [19]:
dynface.task_df.to_csv("reward1_BPD_motionpars.csv")

In [19]:
dynface.task_df

,trans_x_mean,trans_x_max,trans_y_mean,trans_y_max,trans_z_mean,trans_z_max,rot_x_mean,rot_x_max,rot_y_mean,rot_y_max,rot_z_mean,rot_z_max,framewise_displacement_mean,framewise_displacement_max
sub-50129,-0.108557,0.077428,0.004888,0.299037,0.184501,1.017690,0.000284,0.013889,0.000185,0.007135,-0.001831,0.002662,0.231932,1.671041
sub-50024,-0.056519,0.059397,0.195545,0.546622,0.045989,0.617673,-0.006293,0.004269,-0.006057,0.000912,-0.000237,0.005386,0.133995,1.305253
sub-50174,-0.016366,0.067294,0.088368,0.337951,-0.027916,0.538612,0.004858,0.010215,-0.002077,0.001266,-0.000908,0.000000,0.105240,0.576728
sub-50023,0.020920,0.137613,-0.009351,0.156905,-0.191751,0.023538,0.002443,0.005713,0.000983,0.003023,-0.001230,0.000196,0.101112,0.360827
sub-50027,-0.017053,0.058092,0.069889,0.208567,0.403542,0.674833,-0.000210,0.002251,-0.002434,0.000643,0.000347,0.001512,0.092931,0.495908
sub-50162,-0.048097,0.000003,0.094388,0.152346,-0.424330,0.018077,0.006008,0.010897,-0.000737,0.000428,-0.000871,0.000101,0.053594,0.118123
sub-50070,-0.110707,0.174075,0.215738,2.179960,-0.184914,2.420050,0.009364,0.088542,0.004744,0.014025,-0.004118,0.000802,0.217867,6.433287
sub-50106,0.079223,0.201422,0.277091,0.545240,0.525200,0.995413,0.003529,0.010794,0.000352,0.002853,0.005018,0.009318,0.119638,1.025588
sub-50008,0.005214,0.088445,0.044572,0.204206,-0.079777,0.145587,0.008213,0.026893,0.001007,0.003148,-0.002391,0.000091,0.104514,1.280773
sub-50104,-0.063257,0.080512,0.228912,0.643828,0.058004,0.725291,0.009141,0.017255,0.001012,0.003802,0.001028,0.004850,0.173979,1.331170


Example 2: Run percent coverage for reward1 NON subjects
- NOTE! This will take several minutes to run

In [20]:
#Use the dataframe we already generated and run the percent coverage script
dynface.percent_coverage('percent_coverage/bnatlas.nii.gz', 'percent_coverage/bnatlas.nii.txt') #Specify the atlas and the labels to use

Preprocessed_ICA/NON/
['percent_coverage/bnatlas.nii.gz', 'Preprocessed_ICA/NON/sub-50002/ses-1/func/sub-50002_ses-1_task-dynface_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed_ICA/NON/sub-50004/ses-1/func/sub-50004_ses-1_task-dynface_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed_ICA/NON/sub-50006/ses-1/func/sub-50006_ses-1_task-dynface_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed_ICA/NON/sub-50007/ses-1/func/sub-50007_ses-1_task-dynface_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed_ICA/NON/sub-50008/ses-1/func/sub-50008_ses-1_task-dynface_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed_ICA/NON/sub-50009/ses-1/func/sub-50009_ses-1_task-dynface_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed_ICA/NON/sub-50010/ses-1/func/sub-50010_ses-1_task-dynface_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocess

/data/D2/data/jupyter-notebooks/QC_Pipeline.py:191: RuntimeWarning: invalid value encountered in true_divide
  avg_signal_strength = signal_strength[0]/n_voxels


In [9]:
display(dynface.percent_coverage_table) #Show the table that was generated

,1 SFG_L_7_1,2 SFG_R_7_1,3 SFG_L_7_2,4 SFG_R_7_2,5 SFG_L_7_3,6 SFG_R_7_3,7 SFG_L_7_4,8 SFG_R_7_4,9 SFG_L_7_5,10 SFG_R_7_5,11 SFG_L_7_6,12 SFG_R_7_6,13 SFG_L_7_7,14 SFG_R_7_7,15 MFG_L_7_1,16 MFG_R_7_1,17 MFG_L_7_2,18 MFG_R_7_2,19 MFG_L_7_3,20 MFG_R_7_3,21 MFG_L_7_4,22 MFG_R_7_4,23 MFG_L_7_5,24 MFG_R_7_5,25 MFG_L_7_6,26 MFG_R_7_6,27 MFG_L_7_7,28 MFG_R_7_7,29 IFG_L_6_1,30 IFG_R_6_1,31 IFG_L_6_2,32 IFG_R_6_2,33 IFG_L_6_3,34 IFG_R_6_3,35 IFG_L_6_4,36 IFG_R_6_4,37 IFG_L_6_5,38 IFG_R_6_5,39 IFG_L_6_6,40 IFG_R_6_6,41 OrG_L_6_1,42 OrG_R_6_1,43 OrG_L_6_2,44 OrG_R_6_2,45 OrG_L_6_3,46 OrG_R_6_3,47 OrG_L_6_4,48 OrG_R_6_4,49 OrG_L_6_5,50 OrG_R_6_5,51 OrG_L_6_6,52 OrG_R_6_6,53 PrG_L_6_1,54 PrG_R_6_1,55 PrG_L_6_2,56 PrG_R_6_2,57 PrG_L_6_3,58 PrG_R_6_3,59 PrG_L_6_4,60 PrG_R_6_4,61 PrG_L_6_5,62 PrG_R_6_5,63 PrG_L_6_6,64 PrG_R_6_6,65 PCL_L_2_1,66 PCL_R_2_1,67 PCL_L_2_2,68 PCL_R_2_2,69 STG_L_6_1,70 STG_R_6_1,71 STG_L_6_2,72 STG_R_6_2,73 STG_L_6_3,74 STG_R_6_3,75 STG_L_6_4,76 STG_R_6_4,77 STG_L_6_5,78 STG_R_6_5,79 STG_L_6_6,80 STG_R_6_6,81 MTG_L_4_1,82 MTG_R_4_1,83 MTG_L_4_2,84 MTG_R_4_2,85 MTG_L_4_3,86 MTG_R_4_3,87 MTG_L_4_4,88 MTG_R_4_4,89 ITG_L_7_1,90 ITG_R_7_1,91 ITG_L_7_2,92 ITG_R_7_2,93 ITG_L_7_3,94 ITG_R_7_3,95 ITG_L_7_4,96 ITG_R_7_4,97 ITG_L_7_5,98 ITG_R_7_5,99 ITG_L_7_6,100 ITG_R_7_6,101 ITG_L_7_7,102 ITG_R_7_7,103 FuG_L_3_1,104 FuG_R_3_1,105 FuG_L_3_2,106 FuG_R_3_2,107 FuG_L_3_3,108 FuG_R_3_3,109 PhG_L_6_1,110 PhG_R_6_1,111 PhG_L_6_2,112 PhG_R_6_2,113 PhG_L_6_3,114 PhG_R_6_3,115 PhG_L_6_4,116 PhG_R_6_4,117 PhG_L_6_5,118 PhG_R_6_5,119 PhG_L_6_6,120 PhG_R_6_6,121 pSTS_L_2_1,122 pSTS_R_2_1,123 pSTS_L_2_2,124 pSTS_R_2_2,125 SPL_L_5_1,126 SPL_R_5_1,127 SPL_L_5_2,128 SPL_R_5_2,129 SPL_L_5_3,130 SPL_R_5_3,131 SPL_L_5_4,132 SPL_R_5_4,133 SPL_L_5_5,134 SPL_R_5_5,135 IPL_L_6_1,136 IPL_R_6_1,137 IPL_L_6_2,138 IPL_R_6_2,139 IPL_L_6_3,140 IPL_R_6_3,141 IPL_L_6_4,142 IPL_R_6_4,143 IPL_L_6_5,144 IPL_R_6_5,145 IPL_L_6_6,146 IPL_R_6_6,147 PCun_L_4_1,148 PCun_R_4_1,149 PCun_L_4_2,150 PCun_R_4_2,151 PCun_L_4_3,152 PCun_R_4_3,153 PCun_L_4_4,154 PCun_R_4_4,155 PoG_L_4_1,156 PoG_R_4_1,157 PoG_L_4_2,158 PoG_R_4_2,159 PoG_L_4_3,160 PoG_R_4_3,161 PoG_L_4_4,162 PoG_R_4_4,163 INS_L_6_1,164 INS_R_6_1,165 INS_L_6_2,166 INS_R_6_2,167 INS_L_6_3,168 INS_R_6_3,169 INS_L_6_4,170 INS_R_6_4,171 INS_L_6_5,172 INS_R_6_5,173 INS_L_6_6,174 INS_R_6_6,175 CG_L_7_1,176 CG_R_7_1,177 CG_L_7_2,178 CG_R_7_2,179 CG_L_7_3,180 CG_R_7_3,181 CG_L_7_4,182 CG_R_7_4,183 CG_L_7_5,184 CG_R_7_5,185 CG_L_7_6,186 CG_R_7_6,187 CG_L_7_7,188 CG_R_7_7,189 Cun_L_5_1,190 Cun_R_5_1,191 Cun_L_5_2,192 Cun_R_5_2,193 Cun_L_5_3,194 Cun_R_5_3,195 Cun_L_5_4,196 Cun_R_5_4,197 Cun_L_5_5,198 Cun_R_5_5,199 OcG_L_4_1,200 OcG_R_4_1,201 OcG_L_4_2,202 OcG_R_4_2,203 OcG_L_4_3,204 OcG_R_4_3,205 OcG_L_4_4,206 OcG_R_4_4,207 sOcG_L_2_1,208 sOcG_R_2_1,209 sOcG_L_2_2,210 sOcG_R_2_2,211 Amyg_L_2_1,212 Amyg_R_2_1,213 Amyg_L_2_2,214 Amyg_R_2_2,215 Hipp_L_2_1,216 Hipp_R_2_1,217 Hipp_L_2_2,218 Hipp_R_2_2,219 Str_L_6_1,220 Str_R_6_1,221 Str_L_6_2,222 Str_R_6_2,223 Str_L_6_3,224 Str_R_6_3,225 Str_L_6_4,226 Str_R_6_4,227 Str_L_6_5,228 Str_R_6_5,229 Str_L_6_6,230 Str_R_6_6,231 Tha_L_8_1,232 Tha_R_8_1,233 Tha_L_8_2,234 Tha_R_8_2,235 Tha_L_8_3,236 Tha_R_8_3,237 Tha_L_8_4,238 Tha_R_8_4,239 Tha_L_8_5,240 Tha_R_8_5,241 Tha_L_8_6,242 Tha_R_8_6,243 Tha_L_8_7,244 Tha_R_8_7,245 Tha_L_8_8,246 Tha_R_8_8
bnatlas.n,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000

In [21]:
dynface.percent_coverage_table.to_csv("Maya_ICA_ALL_PercentCoverage.csv")

In [10]:
dynface.output_low_coverage(0.5) #highlight cells which have output less than 50%

## QC on Pre-processed tasks
- create a list containing all of the task names for your BIDs formatted fmriprep data
- Specify the folders containing the subjects
- generate tables for each dataset and task
- output number subjects flagged for mean and max values

In [10]:
from QC_Pipeline import Task_DataFrame as QC #Import custom module for QC
tasks = ['reward1', 'reward2', 'efnback1', 'efnback2', 'dynface', 'rest']
subject_folders = ['Preprocessed/NON', 'Preprocessed/BPD']

QC_objects = [] #List which we will append the QC objects we create into.  This will allow for indexing later on
coverage_objects = [] #List to store percent coverage table outputs
for folder in subject_folders: #Loop thru Directories containing fmriprep data
    for task in tasks: #Loop thru each task
        task_object = QC(task, folder)
        QC_objects.append(task_object) #Output the new QC object to our list
        percent_coverage_table = task_object.percent_coverage('percent_coverage/bnatlas.nii.gz', 'percent_coverage/bnatlas.nii.txt')
        coverage_objects.append(percent_coverage_table)


reward1 and Preprocessed/NON
Preprocessed/NON
['percent_coverage/bnatlas.nii.gz', 'Preprocessed/NON/sub-50002/ses-1/func/sub-50002_ses-1_task-reward1_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50004/ses-1/func/sub-50004_ses-1_task-reward1_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50006/ses-1/func/sub-50006_ses-1_task-reward1_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50007/ses-1/func/sub-50007_ses-1_task-reward1_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50008/ses-1/func/sub-50008_ses-1_task-reward1_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50009/ses-1/func/sub-50009_ses-1_task-reward1_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50010/ses-1/func/sub-50010_ses-1_task-reward1_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/N

jupyter-notebooks/QC_Pipeline.py:191: RuntimeWarning: invalid value encountered in true_divide
  avg_signal_strength = signal_strength[0]/n_voxels
jupyter-notebooks/QC_Pipeline.py:191: RuntimeWarning: invalid value encountered in true_divide
  avg_signal_strength = signal_strength[0]/n_voxels
jupyter-notebooks/QC_Pipeline.py:191: RuntimeWarning: invalid value encountered in true_divide
  avg_signal_strength = signal_strength[0]/n_voxels
jupyter-notebooks/QC_Pipeline.py:191: RuntimeWarning: invalid value encountered in true_divide
  avg_signal_strength = signal_strength[0]/n_voxels
jupyter-notebooks/QC_Pipeline.py:191: RuntimeWarning: invalid value encountered in true_divide
  avg_signal_strength = signal_strength[0]/n_voxels
jupyter-notebooks/QC_Pipeline.py:191: RuntimeWarning: invalid value encountered in true_divide
  avg_signal_strength = signal_strength[0]/n_voxels


reward2 and Preprocessed/NON
Preprocessed/NON
['percent_coverage/bnatlas.nii.gz', 'Preprocessed/NON/sub-50002/ses-1/func/sub-50002_ses-1_task-reward2_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50004/ses-1/func/sub-50004_ses-1_task-reward2_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50006/ses-1/func/sub-50006_ses-1_task-reward2_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50007/ses-1/func/sub-50007_ses-1_task-reward2_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50008/ses-1/func/sub-50008_ses-1_task-reward2_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50009/ses-1/func/sub-50009_ses-1_task-reward2_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50010/ses-1/func/sub-50010_ses-1_task-reward2_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/N

jupyter-notebooks/QC_Pipeline.py:191: RuntimeWarning: invalid value encountered in true_divide
  avg_signal_strength = signal_strength[0]/n_voxels
jupyter-notebooks/QC_Pipeline.py:191: RuntimeWarning: invalid value encountered in true_divide
  avg_signal_strength = signal_strength[0]/n_voxels
jupyter-notebooks/QC_Pipeline.py:191: RuntimeWarning: invalid value encountered in true_divide
  avg_signal_strength = signal_strength[0]/n_voxels
jupyter-notebooks/QC_Pipeline.py:191: RuntimeWarning: invalid value encountered in true_divide
  avg_signal_strength = signal_strength[0]/n_voxels
jupyter-notebooks/QC_Pipeline.py:191: RuntimeWarning: invalid value encountered in true_divide
  avg_signal_strength = signal_strength[0]/n_voxels


efnback1 and Preprocessed/NON
Preprocessed/NON
['percent_coverage/bnatlas.nii.gz', 'Preprocessed/NON/sub-50002/ses-1/func/sub-50002_ses-1_task-efnback1_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50004/ses-1/func/sub-50004_ses-1_task-efnback1_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50006/ses-1/func/sub-50006_ses-1_task-efnback1_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50007/ses-1/func/sub-50007_ses-1_task-efnback1_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50008/ses-1/func/sub-50008_ses-1_task-efnback1_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50009/ses-1/func/sub-50009_ses-1_task-efnback1_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/NON/sub-50010/ses-1/func/sub-50010_ses-1_task-efnback1_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Prepro

...

efnback2 and Preprocessed/BPD
Preprocessed/BPD
['percent_coverage/bnatlas.nii.gz', 'Preprocessed/BPD/sub-50003/ses-1/func/sub-50003_ses-1_task-efnback2_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50017/ses-1/func/sub-50017_ses-1_task-efnback2_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50025/ses-1/func/sub-50025_ses-1_task-efnback2_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50028/ses-1/func/sub-50028_ses-1_task-efnback2_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50044/ses-1/func/sub-50044_ses-1_task-efnback2_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50048/ses-1/func/sub-50048_ses-1_task-efnback2_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50062/ses-1/func/sub-50062_ses-1_task-efnback2_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Prepro

jupyter-notebooks/QC_Pipeline.py:191: RuntimeWarning: invalid value encountered in true_divide
  avg_signal_strength = signal_strength[0]/n_voxels
jupyter-notebooks/QC_Pipeline.py:191: RuntimeWarning: invalid value encountered in true_divide
  avg_signal_strength = signal_strength[0]/n_voxels


dynface and Preprocessed/BPD
Preprocessed/BPD
['percent_coverage/bnatlas.nii.gz', 'Preprocessed/BPD/sub-50003/ses-1/func/sub-50003_ses-1_task-dynface_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50017/ses-1/func/sub-50017_ses-1_task-dynface_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50025/ses-1/func/sub-50025_ses-1_task-dynface_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50028/ses-1/func/sub-50028_ses-1_task-dynface_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50044/ses-1/func/sub-50044_ses-1_task-dynface_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50048/ses-1/func/sub-50048_ses-1_task-dynface_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50062/ses-1/func/sub-50062_ses-1_task-dynface_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/B

jupyter-notebooks/QC_Pipeline.py:191: RuntimeWarning: invalid value encountered in true_divide
  avg_signal_strength = signal_strength[0]/n_voxels


rest and Preprocessed/BPD
Preprocessed/BPD
['percent_coverage/bnatlas.nii.gz', 'Preprocessed/BPD/sub-50003/ses-1/func/sub-50003_ses-1_task-rest_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50017/ses-1/func/sub-50017_ses-1_task-rest_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50025/ses-1/func/sub-50025_ses-1_task-rest_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50028/ses-1/func/sub-50028_ses-1_task-rest_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50044/ses-1/func/sub-50044_ses-1_task-rest_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50048/ses-1/func/sub-50048_ses-1_task-rest_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50062/ses-1/func/sub-50062_ses-1_task-rest_run-1_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz', 'Preprocessed/BPD/sub-50077/ses-1/func/

## Example 3:
### create a dictionary with each subject and what tasks did not meet the criteria

In [67]:
mean_dict = {}
max_dict = {}
for i, task in enumerate(tasks):
    for index, row in QC_objects[i].mean_table.iterrows(): #check thru each row of mean table. Add in task name to each sub key
        try:
            mean_dict[index].append(QC_objects[i].task) #try to append to a list of the subject
        except KeyError: #if the subject doesn't already have a key, create a key that is a list
            mean_dict[index] = [QC_objects[i].task]
    for index, row in QC_objects[i].max_table.iterrows(): #do the same thing for the max tables
        try:
            max_dict[index].append(QC_objects[i].task)
        except KeyError:
            max_dict[index] = [QC_objects[i].task]
            
for j, task in enumerate(tasks): # do the same thing for the BPD folders which are 6 indeces away
    i = j+6
    for index, row in QC_objects[i].mean_table.iterrows(): #check thru each row of mean table. Add in task name to each sub key
        try:
            mean_dict[index].append(QC_objects[i].task) #try to append to a list of the subject
        except KeyError: #if the subject doesn't already have a key, create a key that is a list
            mean_dict[index] = [QC_objects[i].task]
    for index, row in QC_objects[i].max_table.iterrows(): #do the same thing for the max tables
        try:
            max_dict[index].append(QC_objects[i].task)
        except KeyError:
            max_dict[index] = [QC_objects[i].task]

In [72]:
mean_dict #subject tasks that had an average motion parameter less than 0.5

{'sub-50108': ['reward1',
  'reward2',
  'efnback1',
  'efnback2',
  'dynface',
  'rest'],
 'sub-50054': ['reward1',
  'reward2',
  'efnback1',
  'efnback2',
  'dynface',
  'rest'],
 'sub-50007': ['reward1', 'reward2', 'efnback2', 'dynface', 'rest'],
 'sub-50010': ['reward1'],
 'sub-50078': ['reward1', 'reward2', 'efnback1', 'efnback2', 'rest'],
 'sub-50126': ['reward1', 'reward2'],
 'sub-50183': ['reward1', 'reward2', 'efnback1', 'efnback2', 'dynface'],
 'sub-50111': ['reward1', 'reward2', 'efnback1', 'efnback2'],
 'sub-50018': ['reward1', 'efnback2', 'dynface'],
 'sub-50122': ['reward1', 'reward2', 'efnback1', 'dynface', 'rest'],
 'sub-50581': ['reward1'],
 'sub-50187': ['reward1', 'reward2'],
 'sub-50097': ['reward1'],
 'sub-50004': ['reward1',
  'reward2',
  'efnback1',
  'efnback2',
  'dynface',
  'rest'],
 'sub-50101': ['reward2'],
 'sub-50014': ['reward2'],
 'sub-50119': ['reward2', 'efnback2'],
 'sub-50009': ['reward2', 'dynface', 'rest'],
 'sub-50167': ['reward2'],
 'sub-50081

In [74]:
max_dict #subject tasks that had a max motion parameter less than 5

{'sub-50116': ['reward1', 'rest'],
 'sub-50149': ['reward1', 'reward2', 'efnback2', 'dynface', 'rest'],
 'sub-50141': ['reward1', 'reward2', 'efnback1', 'dynface'],
 'sub-50108': ['reward1',
  'reward2',
  'efnback1',
  'efnback2',
  'dynface',
  'rest'],
 'sub-50054': ['reward1',
  'reward2',
  'efnback1',
  'efnback2',
  'dynface',
  'rest'],
 'sub-50175': ['reward1', 'efnback1', 'dynface'],
 'sub-50007': ['reward1', 'reward2', 'dynface', 'rest'],
 'sub-50010': ['reward1', 'reward2'],
 'sub-50078': ['reward1', 'efnback1', 'efnback2'],
 'sub-50179': ['reward1', 'reward2'],
 'sub-50153': ['reward1', 'efnback1'],
 'sub-50117': ['reward1', 'dynface', 'rest'],
 'sub-50183': ['reward1', 'reward2', 'rest'],
 'sub-50111': ['reward1', 'reward2', 'efnback1', 'rest'],
 'sub-50018': ['reward1', 'efnback2'],
 'sub-50122': ['reward1', 'efnback2', 'rest'],
 'sub-50080': ['reward1'],
 'sub-50581': ['reward1', 'reward2', 'dynface', 'rest'],
 'sub-50127': ['reward1', 'reward2'],
 'sub-50167': ['reward